In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_data = 5000   # number of data
num_vars = 5      # number of decision variables
num_ints = 5      # number of integer decision variables
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_train = np.random.uniform(1.0, 6.0, (train_size, 1)).astype(np.float32)
a_train = np.random.uniform(0.2, 1.2, (train_size, num_vars-1)).astype(np.float32)
p_test = np.random.uniform(1.0, 6.0, (test_size, 1)).astype(np.float32)
a_test = np.random.uniform(0.2, 1.2, (test_size, num_vars-1)).astype(np.float32)
p_dev = np.random.uniform(1.0, 6.0, (val_size, 1)).astype(np.float32)
a_dev = np.random.uniform(0.2, 1.2, (val_size, num_vars-1)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")

In [6]:
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## NM Problem

In [7]:
import neuromancer as nm
from problem.neural import probRosenbrock

def getNMProb(round_module):
    # parameters
    p = nm.constraint.variable("p")
    a = nm.constraint.variable("a")
    # variables
    x_bar = nm.constraint.variable("x_bar")
    x_rnd = nm.constraint.variable("x_rnd")

    # model
    obj_bar, constrs_bar = probRosenbrock(x_bar, p, a, num_vars=num_vars, alpha=100)
    obj_rnd, constrs_rnd = probRosenbrock(x_rnd, p, a, num_vars=num_vars, alpha=100)

    # define neural architecture for the solution mapping
    func = nm.modules.blocks.MLP(insize=num_vars, outsize=num_vars, bias=True,
                                 linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU, hsizes=[80]*4)
    # solution map from model parameters: sol_map(p) -> x
    sol_map = nm.system.Node(func, ["p", "a"], ["x_bar"], name="smap")

    # penalty loss for mapping
    components = [sol_map]
    loss = nm.loss.PenaltyLoss(obj_bar, constrs_bar)
    problem = nm.problem.Problem(components, loss)

    # penalty loss for rounding
    components = [sol_map, round_module]
    loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
    problem_rnd = nm.problem.Problem(components, loss)

    return problem, problem_rnd

## Exact Solver

In [8]:
from problem.solver import exactRosenbrock
model = exactRosenbrock(n_vars=num_vars, n_integers=num_ints)

In [9]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    model.setParamValue(p, *a)
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:38<00:00, 10.16it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      1.786264               0.0      0.097874
std       1.361469               0.0      0.033692
min       0.000000               0.0      0.072185
25%       0.466428               0.0      0.076158
50%       1.731629               0.0      0.090455
75%       2.953879               0.0      0.093186
max       4.199299               0.0      0.260935


In [10]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"[1.0, 0.0, 1.0, 1.0, 0.0]",1.634025,0.0,0.129671
1,"[1.0, 1.0, 1.0, 1.0, 0.0]",0.261138,0.0,0.090487
2,"[1.0, 1.0, 1.0, 0.0, 0.0]",1.244660,0.0,0.077497
3,"[1.0, 0.0, 0.0, 0.0, 0.0]",3.591436,0.0,0.075948
4,"[1.0, 0.0, 0.0, 0.0, 0.0]",4.187316,0.0,0.076479
...,...,...,...,...
995,"[1.0, 0.0, 0.0, 0.0, 0.0]",4.089960,0.0,0.182148
996,"[1.0, 1.0, 1.0, 0.0, 0.0]",1.625817,0.0,0.182140
997,"[1.0, 1.0, 1.0, 1.0, 0.0]",0.959324,0.0,0.105354
998,"[1.0, 1.0, 1.0, 1.0, 0.0]",1.077039,0.0,0.090769


## Heuristic

In [11]:
from heuristic import naive_round

In [12]:
# relaxed model
model_rel = model.relax()

In [13]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    model.setParamValue(p, *a)
    model_rel.setParamValue(p, *a)
    tick = time.time()
    xval_init, _ = model_rel.solve("scip", max_iter=100)
    naive_round(xval_init, model)
    tock = time.time()
    xval, objval = model.getVal()
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [14:25<00:00,  1.16it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      0.490161          1.027246      0.864290
std       0.687427          0.742127      0.213636
min       0.000000          0.000000      0.435925
25%       0.000000          0.266070      0.679214
50%       0.000000          1.083178      0.787029
75%       0.872345          1.644168      0.994702
max       3.176095          2.577283      1.525655


In [14]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"[1, 1, 1, 1, 0]",0.229294,0.501649,0.794145
1,"[1, 1, 1, 1, 1]",0.000000,0.266266,0.791898
2,"[1, 1, 1, 1, 1]",0.000000,1.186666,0.786137
3,"[1, 1, 1, 0, 0]",1.407886,1.583487,0.672993
4,"[1, 1, 1, 1, 0]",1.168298,2.072099,0.777687
...,...,...,...,...
995,"[1, 1, 1, 0, 0]",2.049713,1.962043,0.789336
996,"[1, 1, 1, 1, 1]",0.000000,1.136647,0.771636
997,"[1, 1, 1, 1, 1]",0.000000,0.414065,0.772562
998,"[1, 1, 1, 1, 1]",0.000000,0.274599,0.989056


## Learning to Round

In [15]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.int_ind}, name="round")
_, problem = getNMProb(round_func)

In [16]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 49.70849609375
epoch: 1  train_loss: 41.51247024536133
epoch: 2  train_loss: 40.15127944946289
epoch: 3  train_loss: 50.554386138916016
epoch: 4  train_loss: 45.82106018066406
epoch: 5  train_loss: 49.82112121582031
epoch: 6  train_loss: 40.62741470336914
epoch: 7  train_loss: 39.87377166748047
epoch: 8  train_loss: 47.59299087524414
epoch: 9  train_loss: 42.070716857910156
epoch: 10  train_loss: 41.75148010253906
epoch: 11  train_loss: 37.79279327392578
epoch: 12  train_loss: 46.500038146972656
epoch: 13  train_loss: 39.58802032470703
epoch: 14  train_loss: 45.82360076904297
epoch: 15  train_loss: 49.97172546386719
epoch: 16  train_loss: 31.36939239501953
epoch: 17  train_loss: 40.5133171081543
epoch: 18  train_loss: 38.941890716552734
epoch: 19  train_loss: 26.786985397338867
epoch: 20  train_loss: 28.458574295043945
epoch: 21  train_loss: 25.684894561767578
epoch: 22  train_loss: 28.383136749267578
epoch: 23  train_loss: 26.57742691040039
epoch: 24  train_loss:

In [17]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # eval
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    model.setParamValue(p, *a)
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 253.59it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      4.635183          0.187780      0.003456
std       0.665417          0.358798      0.000792
min       2.391423          0.000000      0.001996
25%       4.182341          0.000000      0.003000
50%       4.613650          0.000000      0.003140
75%       5.032913          0.211223      0.003997
max       8.252681          2.135950      0.006531


In [18]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"(1.0, 0.0, 1.0, 0.0, 1.0)",3.306289,0.000000,0.005077
1,"(1.0, 0.0, 1.0, 0.0, 1.0)",4.530400,0.000000,0.005055
2,"(1.0, 0.0, 1.0, 0.0, 1.0)",4.081789,0.000000,0.005428
3,"(1.0, 0.0, 1.0, 0.0, 0.0)",4.189243,0.583487,0.005495
4,"(1.0, 0.0, 1.0, 0.0, 0.0)",5.372702,0.072099,0.004017
...,...,...,...,...
995,"(1.0, 0.0, 1.0, 0.0, 0.0)",4.807213,0.962043,0.003000
996,"(1.0, 0.0, 1.0, 0.0, 1.0)",5.250714,0.000000,0.002504
997,"(1.0, 0.0, 1.0, 0.0, 1.0)",5.319346,0.000000,0.003005
998,"(1.0, 0.0, 1.0, 0.0, 1.0)",5.160257,0.000000,0.003608


## Learnable Threshold

In [19]:
from model.layer import netFC
from model.threshold import roundThresholdModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x_bar"], output_keys=["x_rnd"],
                                 int_ind={"x_bar":model.int_ind}, name="round")
_, problem = getNMProb(round_func)

In [20]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 57.52654266357422
epoch: 1  train_loss: 45.62474060058594
epoch: 2  train_loss: 36.37605285644531
epoch: 3  train_loss: 43.228328704833984
epoch: 4  train_loss: 41.60516357421875
epoch: 5  train_loss: 41.10173034667969
epoch: 6  train_loss: 45.800567626953125
epoch: 7  train_loss: 45.91948699951172
epoch: 8  train_loss: 54.22880554199219
epoch: 9  train_loss: 53.48687744140625
epoch: 10  train_loss: 53.12178039550781
epoch: 11  train_loss: 58.54276657104492
epoch: 12  train_loss: 62.96484375
epoch: 13  train_loss: 68.48212432861328
epoch: 14  train_loss: 57.43519973754883
epoch: 15  train_loss: 58.23943328857422
epoch: 16  train_loss: 65.05147552490234
epoch: 17  train_loss: 62.89979934692383
epoch: 18  train_loss: 68.83914947509766
epoch: 19  train_loss: 69.84069061279297
epoch: 20  train_loss: 73.07505798339844
epoch: 21  train_loss: 73.09027862548828
epoch: 22  train_loss: 71.87255859375
epoch: 23  train_loss: 69.55891418457031
epoch: 24  train_loss: 85.3338317

In [21]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    model.setParamValue(p, *a)
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 260.54it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     20.475778          1.022164      0.003326
std      12.800103          0.904140      0.000735
min       4.000000          0.000000      0.001996
25%       8.199815          0.180235      0.003000
50%      22.165774          0.749944      0.003005
75%      31.056224          1.703099      0.003595
max      54.667122          3.435206      0.006511


In [22]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"(0.0, 0.0, 1.0, -2.0, 0.0)",20.919400,1.501649,0.006274
1,"(1.0, 0.0, 0.0, -2.0, 0.0)",19.043393,0.266266,0.005022
2,"(1.0, 0.0, 1.0, -1.0, 0.0)",7.815769,0.000000,0.005499
3,"(0.0, 0.0, 0.0, 0.0, 0.0)",4.000000,0.708256,0.006302
4,"(0.0, 0.0, 0.0, -1.0, 0.0)",9.361666,0.000000,0.005533
...,...,...,...,...
995,"(0.0, 0.0, 0.0, 0.0, 0.0)",4.000000,0.518979,0.002995
996,"(1.0, 0.0, 0.0, -1.0, 0.0)",8.155153,0.000000,0.002585
997,"(1.0, 0.0, 0.0, -2.0, 0.0)",29.884134,0.414065,0.003925
998,"(1.0, 0.0, 0.0, -2.0, 0.0)",32.926288,0.274599,0.003003


## Learning to Round with Fixed Solution Mapping

In [23]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.int_ind}, name="round")
problem_rel, problem_rnd = getNMProb(round_func)

In [24]:
# training for mapping
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem_rel.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rel, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 34.12492370605469
epoch: 1  train_loss: 2.0818397998809814
epoch: 2  train_loss: 0.470735639333725
epoch: 3  train_loss: 0.437946081161499
epoch: 4  train_loss: 0.4126089811325073
epoch: 5  train_loss: 0.41550078988075256
epoch: 6  train_loss: 0.3635338246822357
epoch: 7  train_loss: 0.40750646591186523
epoch: 8  train_loss: 0.45146796107292175
epoch: 9  train_loss: 0.40358108282089233
epoch: 10  train_loss: 0.3790501654148102
epoch: 11  train_loss: 0.3917309045791626
epoch: 12  train_loss: 0.37835943698883057
epoch: 13  train_loss: 0.3908189535140991
epoch: 14  train_loss: 0.3323489725589752
epoch: 15  train_loss: 0.40534210205078125
epoch: 16  train_loss: 0.3614099323749542
epoch: 17  train_loss: 0.3596763610839844
epoch: 18  train_loss: 0.352710485458374
epoch: 19  train_loss: 0.3680935800075531
epoch: 20  train_loss: 0.3954349160194397
epoch: 21  train_loss: 0.3657650649547577
epoch: 22  train_loss: 0.39406174421310425
epoch: 23  train_loss: 0.3806690871715545

In [25]:
# freeze sol mapping
#problem_rel.freeze()
# training for rounding
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rnd, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 178.39549255371094
epoch: 1  train_loss: 182.28233337402344
epoch: 2  train_loss: 178.78712463378906
epoch: 3  train_loss: 179.03147888183594
epoch: 4  train_loss: 177.77017211914062
epoch: 5  train_loss: 182.65928649902344
epoch: 6  train_loss: 179.82041931152344
epoch: 7  train_loss: 179.13153076171875
epoch: 8  train_loss: 181.82296752929688
epoch: 9  train_loss: 178.97198486328125
epoch: 10  train_loss: 185.7644805908203
epoch: 11  train_loss: 177.5663604736328
epoch: 12  train_loss: 178.91567993164062
epoch: 13  train_loss: 177.6474609375
epoch: 14  train_loss: 181.13919067382812
epoch: 15  train_loss: 182.2436065673828
epoch: 16  train_loss: 177.83518981933594
epoch: 17  train_loss: 179.97897338867188
epoch: 18  train_loss: 177.72219848632812
epoch: 19  train_loss: 184.91534423828125
epoch: 20  train_loss: 178.2413330078125
epoch: 21  train_loss: 177.55377197265625
epoch: 22  train_loss: 182.93743896484375
epoch: 23  train_loss: 178.30929565429688
epoch: 24 

In [26]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem_rnd(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    model.setParamValue(p, *a)
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 242.63it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      6.605211          1.251305      0.003605
std       5.420684          1.401519      0.001010
min       0.000000          0.000000      0.001999
25%       3.908666          0.392297      0.003000
50%       4.418969          1.000000      0.003503
75%       7.171421          1.430793      0.004000
max      30.283837          8.258777      0.008512


In [27]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"(1.0, 0.0, 0.0, 0.0, 0.0)",3.204402,0.749176,0.006099
1,"(1.0, 1.0, 0.0, 0.0, 0.0)",2.422065,0.366867,0.005522
2,"(0.0, 0.0, 0.0, 0.0, 0.0)",4.000000,1.906667,0.004508
3,"(0.0, 1.0, 0.0, 0.0, 0.0)",4.781358,1.000000,0.005611
4,"(0.0, 1.0, 0.0, 0.0, 0.0)",5.179335,1.000000,0.006524
...,...,...,...,...
995,"(0.0, 0.0, 0.0, 1.0, 0.0)",4.576891,1.000000,0.003509
996,"(1.0, 0.0, 1.0, 0.0, 1.0)",5.250714,0.000000,0.004222
997,"(1.0, 1.0, 1.0, 0.0, 0.0)",1.774578,0.000000,0.003005
998,"(1.0, 1.0, 1.0, 0.0, 0.0)",2.101038,0.000000,0.002506


### Learning to Feasibility Pump

In [28]:
from model.layer import netFC
# define neural architecture for the solution mapping
sol_func = nm.modules.blocks.MLP(insize=num_vars, outsize=num_vars, bias=True,
                                 linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU,
                                 hsizes=[80]*4)
# define neural architecture for rounding
rnd_layer = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)

In [29]:
# function to get nm optimization model
from problem.neural import probRosenbrock
def getProb(x, p, a):
    return probRosenbrock(x, p, a, num_vars=num_vars, alpha=100)

In [30]:
from model import feasibilityPumpModel
# feasibility pump model
problem_rel, problem_fp = feasibilityPumpModel(["p", "a"], getProb, sol_func, rnd_layer, int_ind=model.int_ind, num_iters=3)

In [31]:
# training for mapping
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem_rel.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rel, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 26.019447326660156
epoch: 1  train_loss: 0.5018856525421143
epoch: 2  train_loss: 0.4640052020549774
epoch: 3  train_loss: 0.4077915847301483
epoch: 4  train_loss: 0.39302703738212585
epoch: 5  train_loss: 0.4387400150299072
epoch: 6  train_loss: 0.4374125897884369
epoch: 7  train_loss: 0.4021318256855011
epoch: 8  train_loss: 0.4044865667819977
epoch: 9  train_loss: 0.4192991852760315
epoch: 10  train_loss: 0.34392988681793213
epoch: 11  train_loss: 0.3829454481601715
epoch: 12  train_loss: 0.3520008325576782
epoch: 13  train_loss: 0.39323192834854126
epoch: 14  train_loss: 0.37630024552345276
epoch: 15  train_loss: 0.3270723223686218
epoch: 16  train_loss: 0.4301745295524597
epoch: 17  train_loss: 0.38404619693756104
epoch: 18  train_loss: 0.3138093650341034
epoch: 19  train_loss: 0.37943416833877563
epoch: 20  train_loss: 0.28371384739875793
epoch: 21  train_loss: 0.4230571985244751
epoch: 22  train_loss: 0.3997431695461273
epoch: 23  train_loss: 0.338070750236

In [32]:
# freeze sol mapping
#problem_rel.freeze()
# training for feasibility pump
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_fp, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 1193.126953125
epoch: 1  train_loss: 1210.1878662109375
epoch: 2  train_loss: 1209.5623779296875
epoch: 3  train_loss: 1194.9654541015625
epoch: 4  train_loss: 1191.7130126953125
epoch: 5  train_loss: 1195.9344482421875
epoch: 6  train_loss: 1218.43603515625
epoch: 7  train_loss: 1193.140380859375
epoch: 8  train_loss: 1179.0643310546875
epoch: 9  train_loss: 1194.7115478515625
epoch: 10  train_loss: 1218.4249267578125
epoch: 11  train_loss: 1198.94384765625
epoch: 12  train_loss: 1207.5086669921875
epoch: 13  train_loss: 1213.5775146484375
epoch: 14  train_loss: 1213.363525390625
epoch: 15  train_loss: 1214.5821533203125
epoch: 16  train_loss: 1206.951171875
epoch: 17  train_loss: 1204.026611328125
epoch: 18  train_loss: 1192.2855224609375
epoch: 19  train_loss: 1186.3773193359375
epoch: 20  train_loss: 1199.818115234375
epoch: 21  train_loss: 1199.17138671875
epoch: 22  train_loss: 1203.8134765625
epoch: 23  train_loss: 1206.350341796875
epoch: 24  train_loss: 1

In [33]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem_fp(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    model.setParamValue(p, *a)
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:03<00:00,  8.07it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      6.132644          2.709945      0.121376
std      10.793020          2.556880      0.016714
min       0.000000          0.000000      0.084781
25%       1.509867          0.836753      0.106657
50%       2.872057          1.832545      0.122832
75%       6.214915          3.740688      0.134662
max      86.704107         14.056717      0.199103


In [34]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"(1.0, 1.0, 0.0, 1.0, 1.0)",1.872594,0.501649,0.150280
1,"(1.0, 1.0, 1.0, 1.0, 3.0)",1.044551,8.266266,0.129654
2,"(1.0, 1.0, 1.0, 1.0, 1.0)",0.000000,1.186666,0.134938
3,"(1.0, 1.0, 1.0, 0.0, 1.0)",2.152665,2.583487,0.150752
4,"(1.0, 1.0, 0.0, 1.0, 1.0)",3.185387,2.072099,0.119994
...,...,...,...,...
995,"(0.0, 0.0, 1.0, 1.0, 0.0)",3.194718,0.962043,0.152930
996,"(2.0, 0.0, 0.0, 2.0, 1.0)",24.044279,5.136647,0.133409
997,"(1.0, 2.0, 2.0, 0.0, 2.0)",24.262394,8.414065,0.131910
998,"(1.0, 1.0, 1.0, 1.0, 1.0)",0.000000,0.274599,0.136082
